#### Importando bibliotecas

In [1]:
# desabilita os warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

import plotly.express as px
import matplotlib.pyplot as plt


# from datetime import datetime as dt

#### Tratando dados de solar_wind

##### Impotando o conjunto de dados 

In [3]:
df_solar_wind = pd.read_csv(r'..\data\solar_wind.csv')
df_solar_wind.head(3)

,period,timedelta,bx_gse,by_gse,bz_gse,theta_gse,phi_gse,bx_gsm,by_gsm,bz_gsm,theta_gsm,phi_gsm,bt,density,speed,temperature,source
0,train_a,0 days 00:00:00,-5.55,3.00,1.25,11.09,153.37,-5.55,3.00,1.25,11.09,153.37,6.80,1.53,383.92,110237.0,ac
1,train_a,0 days 00:01:00,-5.58,3.16,1.17,10.10,151.91,-5.58,3.16,1.17,10.10,151.91,6.83,1.69,381.79,123825.0,ac
2,train_a,0 days 00:02:00,-5.15,3.66,0.85,7.87,146.04,-5.15,3.66,0.85,7.87,146.04,6.77,1.97,389.11,82548.0,ac


##### Tratando tipos de dados

In [4]:
df_solar_wind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8392320 entries, 0 to 8392319
Data columns (total 17 columns):
 #   Column       Dtype  
---  ------       -----  
 0   period       object 
 1   timedelta    object 
 2   bx_gse       float64
 3   by_gse       float64
 4   bz_gse       float64
 5   theta_gse    float64
 6   phi_gse      float64
 7   bx_gsm       float64
 8   by_gsm       float64
 9   bz_gsm       float64
 10  theta_gsm    float64
 11  phi_gsm      float64
 12  bt           float64
 13  density      float64
 14  speed        float64
 15  temperature  float64
 16  source       object 
dtypes: float64(14), object(3)
memory usage: 1.1+ GB


In [5]:
# Otimizando tipos de dados das colunas numéricas
col_num_solar_wind = df_solar_wind.select_dtypes(include='float64').columns
df_solar_wind[col_num_solar_wind] = df_solar_wind[col_num_solar_wind].apply(pd.to_numeric, downcast='float')

# Corrigindo tipo de dado da coluna timedelta
df_solar_wind["timedelta"] = pd.to_timedelta(df_solar_wind["timedelta"])

# Corrigindo tipo de dado das colunas categoricas
col_cat_solar_wind = df_solar_wind.select_dtypes(include='object').columns
df_solar_wind[col_cat_solar_wind] = df_solar_wind[col_cat_solar_wind].astype('category')
df_solar_wind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8392320 entries, 0 to 8392319
Data columns (total 17 columns):
 #   Column       Dtype          
---  ------       -----          
 0   period       category       
 1   timedelta    timedelta64[ns]
 2   bx_gse       float32        
 3   by_gse       float32        
 4   bz_gse       float32        
 5   theta_gse    float32        
 6   phi_gse      float32        
 7   bx_gsm       float32        
 8   by_gsm       float32        
 9   bz_gsm       float32        
 10  theta_gsm    float32        
 11  phi_gsm      float32        
 12  bt           float32        
 13  density      float32        
 14  speed        float32        
 15  temperature  float32        
 16  source       category       
dtypes: category(2), float32(14), timedelta64[ns](1)
memory usage: 528.2 MB


In [6]:
# Exportando solar_wind com tipos de dados tratados
df_solar_wind.to_parquet(r'..\data\df_solar_wind.parquet')

##### Tratando Valores fora de escala

In [7]:
# bx_gse: -200 a +200 nT ok
# by_gse: -200 a +200 nT ok
# bz_gse: -200 a +200 nT ok
# theta_gse: -90° a 90° ok
# phi_gse: 0° a 360° ok
# bx_gsm: -200 a +200 nT ok
# by_gsm: -200 a +200 nT ok
# bz_gsm: -200 a +200 nT ok
# theta_gsm: -90° a 90° ok
# phi_gsm: 0° a 360° ok
# bt: 0 a 200 nT ok
# density: 0 a 200 N/cm³ ok 
# speed: 200 a 2.000 km/s speed=0?
# temperature: 10.000 K a 10.000.000 K temperature=0?
df_solar_wind.describe()

,timedelta,bx_gse,by_gse,bz_gse,theta_gse,phi_gse,bx_gsm,by_gsm,bz_gsm,theta_gsm,phi_gsm,bt,density,speed,temperature
count,8392320,8.066432e+06,8.066432e+06,8.066432e+06,8.066432e+06,8.065932e+06,8.066432e+06,8.066432e+06,8.066432e+06,8.066432e+06,8.065932e+06,8.066432e+06,7.707430e+06,7.702765e+06,7.580552e+06
mean,1044 days 16:41:54.955388112,-6.610162e-01,1.097870e-01,-2.274230e-02,1.545628e-01,1.941759e+02,-6.608664e-01,8.842117e-02,-2.912839e-02,1.967691e-01,1.943713e+02,5.606227e+00,4.421726e+00,4.305854e+02,1.150967e+05
std,660 days 08:03:01.062266016,3.669730e+00,3.975340e+00,3.351972e+00,3.212916e+01,9.662813e+01,3.667605e+00,3.909885e+00,3.430580e+00,3.327889e+01,9.682818e+01,3.110002e+00,4.331911e+00,1.005777e+02,1.203120e+05
min,0 days 00:00:00,-5.463000e+01,-5.169000e+01,-5.500000e+01,-8.989000e+01,0.000000e+00,-5.463000e+01,-5.268000e+01,-5.569000e+01,-8.991000e+01,0.000000e+00,3.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00
25%,485 days 15:59:45,-3.270000e+00,-2.460000e+00,-1.700000e+00,-2.145000e+01,1.240700e+02,-3.270000e+00,-2.380000e+00,-1.800000e+00,-2.279000e+01,1.246900e+02,3.640000e+00,1.790000e+00,3.568000e+02,3.978700e+04
50%,971 days 07:59:30,-6.700000e-01,1.100000e-01,1.000000e-02,1.400000e-01,1.768600e+02,-6.700000e-01,8.000000e-02,0.000000e+00,6.000000e-02,1.776600e+02,4.950000e+00,3.340000e+00,4.097000e+02,7.737100e+04
75%,1584 days 23:59:15,2.020000e+00,2.640000e+00,1.690000e+00,2.168000e+01,2.864200e+02,2.020000e+00,2.540000e+00,1.790000e+00,2.313000e+01,2.865600e+02,6.720000e+00,5.710000e+00,4.856600e+02,1.512200e+05
max,2435 days 23:59:00,5.555000e+01,5.760000e+01,7.403000e+01,8.994000e+01,3.600000e+02,5.554000e+01,5.388000e+01,7.245000e+01,8.996000e+01,3.600000e+02,8.053000e+01,1.997000e+02,1.198490e+03,6.223700e+06


In [8]:
# Removendo valores fora discrepantes
df_solar_wind["speed"].loc[(df_solar_wind["speed"] < 200) | (df_solar_wind["speed"] > 2000)] = np.nan
df_solar_wind["temperature"].loc[(df_solar_wind["temperature"] < 10_000) | (df_solar_wind["temperature"] > 10_000_000)] = np.nan

In [9]:
df_solar_wind["source"].unique()

['ac', NaN, 'ds']
Categories (2, object): ['ac', 'ds']

#### Tratando dados de satellite_pos

##### Impotando o conjunto de dados 

In [10]:
df_satellite_pos = pd.read_csv(r'..\data\satellite_pos.csv')
df_satellite_pos.head(3)

,period,timedelta,gse_x_ace,gse_y_ace,gse_z_ace,gse_x_dscovr,gse_y_dscovr,gse_z_dscovr
0,train_a,0 days,1522376.9,143704.6,149496.7,NaN,NaN,NaN
1,train_a,1 days,1525410.9,136108.8,151034.1,NaN,NaN,NaN
2,train_a,2 days,1528484.9,128470.5,152387.7,NaN,NaN,NaN


##### Tratando tipos de dados

In [11]:
df_satellite_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5828 entries, 0 to 5827
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   period        5828 non-null   object 
 1   timedelta     5828 non-null   object 
 2   gse_x_ace     5828 non-null   float64
 3   gse_y_ace     5828 non-null   float64
 4   gse_z_ace     5828 non-null   float64
 5   gse_x_dscovr  1034 non-null   float64
 6   gse_y_dscovr  1034 non-null   float64
 7   gse_z_dscovr  1034 non-null   float64
dtypes: float64(6), object(2)
memory usage: 364.4+ KB


In [12]:
# Otimizando tipos de dados das colunas numéricas
col_num_satellite_pos = df_satellite_pos.select_dtypes(include='float64').columns
df_satellite_pos[col_num_satellite_pos] = df_satellite_pos[col_num_satellite_pos].apply(pd.to_numeric, downcast='float')

# Corrigindo tipo de dado da coluna timedelta
df_satellite_pos["timedelta"] = pd.to_timedelta(df_satellite_pos["timedelta"])

col_cat_satellite_pos = df_satellite_pos.select_dtypes(include='object').columns
df_satellite_pos[col_cat_satellite_pos] = df_solar_wind[col_cat_satellite_pos].astype('category')
df_satellite_pos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5828 entries, 0 to 5827
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype          
---  ------        --------------  -----          
 0   period        5828 non-null   category       
 1   timedelta     5828 non-null   timedelta64[ns]
 2   gse_x_ace     5828 non-null   float64        
 3   gse_y_ace     5828 non-null   float64        
 4   gse_z_ace     5828 non-null   float64        
 5   gse_x_dscovr  1034 non-null   float32        
 6   gse_y_dscovr  1034 non-null   float32        
 7   gse_z_dscovr  1034 non-null   float32        
dtypes: category(1), float32(3), float64(3), timedelta64[ns](1)
memory usage: 256.4 KB


In [13]:
# Exportando df_satellite_pos com tipos de dados tratados
df_solar_wind.to_parquet(r'..\data\df_satellite_pos.parquet')

#### Tratando dados de sunspots

##### Impotando o conjunto de dados 

In [14]:
df_sunspots = pd.read_csv(r'..\data\sunspots.csv')
df_sunspots.head(3)

,period,timedelta,smoothed_ssn
0,train_a,0 days 00:00:00,65.4
1,train_a,13 days 00:00:00,72.0
2,train_a,44 days 00:00:00,76.9


##### Tratando tipos de dados

In [15]:
df_sunspots.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   period        192 non-null    object 
 1   timedelta     192 non-null    object 
 2   smoothed_ssn  192 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.6+ KB


In [16]:
# otimizando tipos de dados das colunas numéricas
col_num_sunspots = df_sunspots.select_dtypes(include='float64').columns
df_sunspots[col_num_sunspots] = df_sunspots[col_num_sunspots].apply(pd.to_numeric, downcast='float')

# Corrigindo tipo de dado da coluna timedelta
df_sunspots["timedelta"] = pd.to_timedelta(df_sunspots["timedelta"])

# Corrigindo tipo de dado da coluna period
df_sunspots["period"] = df_sunspots["period"].astype('category')
df_sunspots.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype          
---  ------        --------------  -----          
 0   period        192 non-null    category       
 1   timedelta     192 non-null    timedelta64[ns]
 2   smoothed_ssn  192 non-null    float32        
dtypes: category(1), float32(1), timedelta64[ns](1)
memory usage: 2.7 KB


In [17]:
# Exportando df_sunspots com tipos de dados tratados
df_solar_wind.to_parquet(r'..\data\df_sunspots.parquet')

#### Tratando dados de labels

##### Impotando o conjunto de dados 

In [18]:
df_labels = pd.read_csv(r'..\data\labels.csv')
df_labels.head(3)

,period,timedelta,dst
0,train_a,0 days 00:00:00,-7
1,train_a,0 days 01:00:00,-10
2,train_a,0 days 02:00:00,-10


##### Tratando tipos de dados

In [19]:
df_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139872 entries, 0 to 139871
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   period     139872 non-null  object
 1   timedelta  139872 non-null  object
 2   dst        139872 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 3.2+ MB


In [20]:
# otimizando tipos de dados das colunas numéricas
col_num_labels = df_labels.select_dtypes(include='int64').columns
df_labels[col_num_labels] = df_labels[col_num_labels].apply(pd.to_numeric, downcast='integer')

# Corrigindo tipo de dado da coluna timedelta
df_labels["timedelta"] = pd.to_timedelta(df_labels["timedelta"])

# Corrigindo tipo de dado da coluna period
df_labels["period"] = df_labels["period"].astype('category')
df_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139872 entries, 0 to 139871
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype          
---  ------     --------------   -----          
 0   period     139872 non-null  category       
 1   timedelta  139872 non-null  timedelta64[ns]
 2   dst        139872 non-null  int16          
dtypes: category(1), int16(1), timedelta64[ns](1)
memory usage: 1.5 MB


In [21]:
# Exportando df_labels com tipos de dados tratados
df_solar_wind.to_parquet(r'..\data\df_labels.parquet')